In [1]:
import scipy
import pandas as pd
import os
from scipy.io import loadmat
import numpy as np
import math 
import collections
import statistics
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn import metrics
# import mrmr
# from mrmr import mrmr_classif

path = "/Users/dhruv/OneDrive - Georgia Institute of Technology/KiTS Project/Files"
training_ids = loadmat(os.path.join(path,'trainingGroup.mat'))

path2 = "/Users/dhruv/OneDrive - Georgia Institute of Technology/KiTS Project/Files"
testing_ids = loadmat(os.path.join(path2,'testingGroup.mat'))

In [2]:
path = "/Users/dhruv/OneDrive - Georgia Institute of Technology/KiTS Project/Files"
features = loadmat(os.path.join(path,'ROIstats_Tumor_5mm.mat'))
roi_stats_tumor = pd.DataFrame(features['ROIstats_Tumor'])

In [3]:
# Check whether data from email is the same as in dataframe
# path = "C:/Users/dhruv/OneDrive - Georgia Institute of Technology/KiTS Project/TextureFeatures/Tumor"
path = "/Users/dhruv/OneDrive - Georgia Institute of Technology/KiTS Project/TextureFeatures/Tumor"
# case = pd.DataFrame(loadmat(os.path.join(path, 'case_00000.mat'))['patientFeats'])
# case.describe()
ogStats = pd.DataFrame(index=range(300), columns=range(196)) 
for filename in os.listdir(path):
#     print(int(filename[5:10]))
    case = pd.DataFrame(loadmat(os.path.join(path, filename))['patientFeats'])
    sList = []
    for i in case.columns:
        sList.append(case[i].mean())
        sList.append(case[i].std())
        sList.append(case[i].skew())
        sList.append(case[i].kurtosis())
    ogStats.loc[int(filename[5:10])] = sList
ogStats

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,195
0,0.410841,0.100405,-0.313258,-0.752497,0.407851,0.088581,-0.493569,-0.755417,0.408477,0.091009,...,-0.378374,-0.789349,0.388178,0.098651,-0.378374,-0.789349,0.388178,0.098651,-0.378374,-0.789349
1,0.346387,0.05336,0.170914,0.973758,0.343953,0.045768,0.111171,0.600018,0.344606,0.045859,...,0.219822,0.757148,0.308604,0.052305,0.219822,0.757148,0.308604,0.052305,0.219822,0.757148
2,0.333582,0.052416,0.43467,1.984541,0.332697,0.049199,0.420148,1.437682,0.332892,0.049375,...,0.453566,1.906616,0.285421,0.052542,0.453566,1.906616,0.285421,0.052542,0.453566,1.906616
3,0.376179,0.066089,-0.077042,0.446638,0.373215,0.057342,-0.103612,0.283139,0.374155,0.058293,...,0.056391,0.26744,0.340293,0.064402,0.056391,0.26744,0.340293,0.064402,0.056391,0.26744
4,0.344729,0.051963,-0.877534,1.203404,0.34264,0.048641,-1.006341,1.136111,0.343222,0.048897,...,-0.598407,0.898564,0.327458,0.061096,-0.598407,0.898564,0.327458,0.061096,-0.598407,0.898564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.354117,0.050059,-0.510155,0.766049,0.352505,0.036562,-0.935184,1.31874,0.353009,0.037335,...,-0.284181,0.711503,0.354506,0.052795,-0.284181,0.711503,0.354506,0.052795,-0.284181,0.711503
296,0.354063,0.030159,0.246317,-0.501566,0.354455,0.026401,0.316825,-0.533231,0.354334,0.027022,...,-0.384496,-0.779786,0.32518,0.056249,-0.384496,-0.779786,0.32518,0.056249,-0.384496,-0.779786
297,0.353108,0.055919,-1.271877,1.110887,0.352655,0.052759,-1.299167,1.00762,0.352871,0.053363,...,-1.297763,1.03593,0.352878,0.053832,-1.297763,1.03593,0.352878,0.053832,-1.297763,1.03593
298,0.37213,0.078054,0.516219,0.000629,0.370329,0.066301,0.465069,-0.337672,0.370441,0.067536,...,0.671959,0.619403,0.359911,0.081921,0.671959,0.619403,0.359911,0.081921,0.671959,0.619403


In [4]:
# Add peritumoural features
path = "/Users/dhruv/OneDrive - Georgia Institute of Technology/KiTS Project/TextureFeatures/InsideKidney/0-3mm"
newStats = pd.DataFrame(index=range(300), columns=range(196)) 
for filename in os.listdir(path):
    case = pd.DataFrame(loadmat(os.path.join(path, filename))['patientFeats'])
    sList = []
    for i in case.columns:
        sList.append(case[i].mean())
        sList.append(case[i].std())
        sList.append(case[i].skew())
        sList.append(case[i].kurtosis())
    newStats.loc[int(filename[5:10])] = sList
allStats = pd.concat([ogStats, newStats], axis=1)
allStats.columns = range(len(allStats.axes[1]))
allStats.to_excel(r"\Users\dhruv\Desktop\AI Research\combinedDataframe.xlsx", index=False)
allStats

,0,1,2,3,4,5,6,7,8,9,...,382,383,384,385,386,387,388,389,390,391
0,0.410841,0.100405,-0.313258,-0.752497,0.407851,0.088581,-0.493569,-0.755417,0.408477,0.091009,...,-0.65154,-0.548728,0.385283,0.083743,-0.65154,-0.548728,0.385283,0.083743,-0.65154,-0.548728
1,0.346387,0.05336,0.170914,0.973758,0.343953,0.045768,0.111171,0.600018,0.344606,0.045859,...,-0.408047,-1.137811,0.333622,0.092492,-0.408047,-1.137811,0.333622,0.092492,-0.408047,-1.137811
2,0.333582,0.052416,0.43467,1.984541,0.332697,0.049199,0.420148,1.437682,0.332892,0.049375,...,-0.601228,-1.021963,0.374188,0.10675,-0.601228,-1.021963,0.374188,0.10675,-0.601228,-1.021963
3,0.376179,0.066089,-0.077042,0.446638,0.373215,0.057342,-0.103612,0.283139,0.374155,0.058293,...,-0.483291,-0.553683,0.376041,0.082541,-0.483291,-0.553683,0.376041,0.082541,-0.483291,-0.553683
4,0.344729,0.051963,-0.877534,1.203404,0.34264,0.048641,-1.006341,1.136111,0.343222,0.048897,...,-0.301653,0.892964,0.311558,0.06143,-0.301653,0.892964,0.311558,0.06143,-0.301653,0.892964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.354117,0.050059,-0.510155,0.766049,0.352505,0.036562,-0.935184,1.31874,0.353009,0.037335,...,-0.383577,-0.353312,0.340273,0.064371,-0.383577,-0.353312,0.340273,0.064371,-0.383577,-0.353312
296,0.354063,0.030159,0.246317,-0.501566,0.354455,0.026401,0.316825,-0.533231,0.354334,0.027022,...,-0.502118,-0.553132,0.361137,0.048458,-0.502118,-0.553132,0.361137,0.048458,-0.502118,-0.553132
297,0.353108,0.055919,-1.271877,1.110887,0.352655,0.052759,-1.299167,1.00762,0.352871,0.053363,...,-0.349421,-0.570594,0.332334,0.05819,-0.349421,-0.570594,0.332334,0.05819,-0.349421,-0.570594
298,0.37213,0.078054,0.516219,0.000629,0.370329,0.066301,0.465069,-0.337672,0.370441,0.067536,...,0.998073,1.838918,0.341282,0.103961,0.998073,1.838918,0.341282,0.103961,0.998073,1.838918


In [15]:
# Load the stored excel file
allStats = pd.read_excel(r'\Users\dhruv\Desktop\AI Research\combinedDataframe.xlsx')
allStats

,0,1,2,3,4,5,6,7,8,9,...,382,383,384,385,386,387,388,389,390,391
0,0.410841,0.100405,-0.313258,-0.752497,0.407851,0.088581,-0.493569,-0.755417,0.408477,0.091009,...,-0.651540,-0.548728,0.385283,0.083743,-0.651540,-0.548728,0.385283,0.083743,-0.651540,-0.548728
1,0.346387,0.053360,0.170914,0.973758,0.343953,0.045768,0.111171,0.600018,0.344606,0.045859,...,-0.408047,-1.137811,0.333622,0.092492,-0.408047,-1.137811,0.333622,0.092492,-0.408047,-1.137811
2,0.333582,0.052416,0.434670,1.984541,0.332697,0.049199,0.420148,1.437682,0.332892,0.049375,...,-0.601228,-1.021963,0.374188,0.106750,-0.601228,-1.021963,0.374188,0.106750,-0.601228,-1.021963
3,0.376179,0.066089,-0.077042,0.446638,0.373215,0.057342,-0.103612,0.283139,0.374155,0.058293,...,-0.483291,-0.553683,0.376041,0.082541,-0.483291,-0.553683,0.376041,0.082541,-0.483291,-0.553683
4,0.344729,0.051963,-0.877534,1.203404,0.342640,0.048641,-1.006341,1.136111,0.343222,0.048897,...,-0.301653,0.892964,0.311558,0.061430,-0.301653,0.892964,0.311558,0.061430,-0.301653,0.892964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.354117,0.050059,-0.510155,0.766049,0.352505,0.036562,-0.935184,1.318740,0.353009,0.037335,...,-0.383577,-0.353312,0.340273,0.064371,-0.383577,-0.353312,0.340273,0.064371,-0.383577,-0.353312
296,0.354063,0.030159,0.246317,-0.501566,0.354455,0.026401,0.316825,-0.533231,0.354334,0.027022,...,-0.502118,-0.553132,0.361137,0.048458,-0.502118,-0.553132,0.361137,0.048458,-0.502118,-0.553132
297,0.353108,0.055919,-1.271877,1.110887,0.352655,0.052759,-1.299167,1.007620,0.352871,0.053363,...,-0.349421,-0.570594,0.332334,0.058190,-0.349421,-0.570594,0.332334,0.058190,-0.349421,-0.570594
298,0.372130,0.078054,0.516219,0.000629,0.370329,0.066301,0.465069,-0.337672,0.370441,0.067536,...,0.998073,1.838918,0.341282,0.103961,0.998073,1.838918,0.341282,0.103961,0.998073,1.838918


In [16]:
# Create training matrix
training_matrix = pd.DataFrame(columns=allStats.columns)
train_ids = (training_ids["trainingGroup"]).flatten()

for train_id in train_ids:
    training_matrix.loc[len(training_matrix.index)] = allStats.loc[train_id-1]

training_matrix

,0,1,2,3,4,5,6,7,8,9,...,382,383,384,385,386,387,388,389,390,391
0,0.429725,0.085325,-1.253611,0.934438,0.428434,0.083179,-1.330143,1.002477,0.428884,0.083625,...,-0.205138,-0.269862,0.384609,0.116490,-0.205138,-0.269862,0.384609,0.116490,-0.205138,-0.269862
1,0.331688,0.040704,-0.301121,1.680213,0.329803,0.035127,-0.532560,1.523264,0.330410,0.034916,...,-0.666184,-0.288588,0.376538,0.071770,-0.666184,-0.288588,0.376538,0.071770,-0.666184,-0.288588
2,0.455426,0.117479,-0.285385,-0.611685,0.450953,0.097248,-0.167481,-0.649576,0.452191,0.101632,...,0.069517,-1.273329,0.373030,0.136027,0.069517,-1.273329,0.373030,0.136027,0.069517,-1.273329
3,0.407505,0.080076,-0.435072,-0.157929,0.405853,0.070332,-0.527916,-0.079006,0.406258,0.071947,...,-0.405388,-0.517457,0.357707,0.082841,-0.405388,-0.517457,0.357707,0.082841,-0.405388,-0.517457
4,0.404259,0.044361,-1.448175,3.067505,0.403723,0.038110,-1.595820,3.545971,0.404102,0.039176,...,-0.145694,0.410215,0.353733,0.076325,-0.145694,0.410215,0.353733,0.076325,-0.145694,0.410215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.343921,0.043464,-0.290737,3.749277,0.343722,0.040714,-0.395807,2.311454,0.343798,0.041187,...,-0.155488,-0.357661,0.326966,0.069204,-0.155488,-0.357661,0.326966,0.069204,-0.155488,-0.357661
196,0.371298,0.058435,-0.634080,0.755627,0.368600,0.050424,-0.864413,0.719898,0.369364,0.051040,...,-0.667796,-0.069800,0.319582,0.058693,-0.667796,-0.069800,0.319582,0.058693,-0.667796,-0.069800
197,0.346442,0.050859,0.294748,-0.872808,0.345726,0.047084,0.264355,-1.018365,0.345919,0.047916,...,0.002388,-0.603891,0.339980,0.087680,0.002388,-0.603891,0.339980,0.087680,0.002388,-0.603891
198,0.389627,0.043350,-1.221438,3.502341,0.389464,0.034193,-1.806318,5.099455,0.389564,0.034682,...,-1.128538,1.523388,0.353703,0.050501,-1.128538,1.523388,0.353703,0.050501,-1.128538,1.523388


In [17]:
# Create testing matrix
testing_matrix = pd.DataFrame(columns=allStats.columns)
test_ids = (testing_ids["testingGroup"]).flatten()

for test_id in test_ids:
    testing_matrix.loc[len(testing_matrix.index)] = allStats.loc[test_id-1]

testing_matrix

,0,1,2,3,4,5,6,7,8,9,...,382,383,384,385,386,387,388,389,390,391
0,0.410841,0.100405,-0.313258,-0.752497,0.407851,0.088581,-0.493569,-0.755417,0.408477,0.091009,...,-0.651540,-0.548728,0.385283,0.083743,-0.651540,-0.548728,0.385283,0.083743,-0.651540,-0.548728
1,0.346387,0.053360,0.170914,0.973758,0.343953,0.045768,0.111171,0.600018,0.344606,0.045859,...,-0.408047,-1.137811,0.333622,0.092492,-0.408047,-1.137811,0.333622,0.092492,-0.408047,-1.137811
2,0.344729,0.051963,-0.877534,1.203404,0.342640,0.048641,-1.006341,1.136111,0.343222,0.048897,...,-0.301653,0.892964,0.311558,0.061430,-0.301653,0.892964,0.311558,0.061430,-0.301653,0.892964
3,0.353593,0.065665,-0.852719,-0.047498,0.352138,0.062585,-0.866827,-0.167973,0.352608,0.063154,...,-0.363594,-1.383228,0.288868,0.101806,-0.363594,-1.383228,0.288868,0.101806,-0.363594,-1.383228
4,0.370433,0.044183,-1.007604,1.742997,0.369965,0.039911,-1.087470,1.823192,0.370219,0.040586,...,-0.892033,0.677909,0.348827,0.056110,-0.892033,0.677909,0.348827,0.056110,-0.892033,0.677909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.390084,0.054458,-1.326432,2.114190,0.389333,0.049726,-1.416686,2.250316,0.389753,0.050603,...,-0.914567,0.046531,0.377126,0.077510,-0.914567,0.046531,0.377126,0.077510,-0.914567,0.046531
96,0.307966,0.050457,0.221142,1.744889,0.306211,0.047551,0.140879,1.365098,0.306670,0.047616,...,-0.030609,-0.710116,0.417111,0.144275,-0.030609,-0.710116,0.417111,0.144275,-0.030609,-0.710116
97,0.315991,0.056214,-0.192709,0.475891,0.315276,0.054041,-0.193059,0.423628,0.315473,0.054327,...,-0.465069,-1.257241,0.315816,0.099042,-0.465069,-1.257241,0.315816,0.099042,-0.465069,-1.257241
98,0.317711,0.030634,-1.774320,4.230097,0.317378,0.027659,-2.038979,4.831865,0.317525,0.027903,...,-0.570857,-0.588073,0.290678,0.045411,-0.570857,-0.588073,0.290678,0.045411,-0.570857,-0.588073


In [24]:
datas = pd.read_excel("/Users/dhruv/Desktop/AI Research/KiTS21_ClinicalData.csv")

datas['ISUP'] = datas['ISUP'].replace([1.0, 2.0], 0)
datas['ISUP'] = datas['ISUP'].replace([3.0, 4.0], 1)
datas = datas.fillna(0)

datas['T-grade'] = datas['T-grade'].replace(['1a', '1b', '2a', '2b'], 0)
datas['T-grade'] = datas['T-grade'].replace([3, 4], 1)
datas = datas.fillna(0)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
# print((datas['T-grade'] == 1).sum())
datas

In [ ]:
train_inds = [num - 1 for num in (training_ids["trainingGroup"]).flatten()]
test_inds = [num - 1 for num in (testing_ids["testingGroup"]).flatten()]

trainISUP = datas['ISUP'].iloc[train_inds].squeeze()
testISUP = datas['ISUP'].iloc[test_inds].squeeze()

trainTgrade = datas['T-grade'].iloc[train_inds].squeeze()
testTgrade = datas['T-grade'].iloc[test_inds].squeeze()

In [ ]:
# def cvClassify(feats,label):
#     n_runs = 20
#     k = 3

#     auc_cv = []
#     feats_cv = []

#     for r in range(n_runs):

#         kf = KFold(n_splits=k, shuffle=True)

#         for train, test in kf.split(feats):
#             feats_train = feats.loc[train, :]
#             label_train = label.loc[train]

#             feats_test = feats.loc[test, :]
#             label_test = label.loc[test]

#             # K = 10% of # rows of feats
#             select_feats = mrmr.mrmr_classif(X=feats_train, y=label_train, K=int(round(len(feats_train.index) / 10)))
            
#             feats_cv.extend(select_feats)

#             feats_train = feats_train.loc[:,select_feats]
#             feats_test = feats_test.loc[:,select_feats]

#             feats_train_ = feats_train.to_numpy()
#             label_train_ = label_train.to_numpy()

#             feats_test_ = feats_test.to_numpy()
#             label_test_ = label_test.to_numpy()

#             model = SVC(gamma='auto', probability=True)
#             model.fit(feats_train,label_train)

#             probs_test = model.predict_proba(feats_test_)
#             fpr, tpr, thresholds = metrics.roc_curve(label_test_, probs_test[:,1], pos_label=1)
#             auc_cv.append(metrics.auc(fpr,tpr))


#     counter = collections.Counter(feats_cv)

#     top_feats_sorted = np.array(counter.most_common(5))
#     top_feats = top_feats_sorted[:,0]

#     final_feats = feats.loc[:, top_feats]
#     final_feats = final_feats.to_numpy()
#     label = label.to_numpy()

#     final_model = SVC(gamma='auto', probability=True)
#     final_model.fit(final_feats,label)

#     return statistics.mean(auc_cv), final_model, top_feats

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

def cvClassify(feats,label):
    n_runs = 3
    k = 10

    auc_cv = []
    feats_cv = []

    for r in range(n_runs):
        
        kf = KFold(n_splits=k, shuffle=True)
        
        for train, test in kf.split(feats):
            feats_train = feats.iloc[train, :]
            label_train = label.iloc[train]

            feats_test = feats.iloc[test, :]
            label_test = label.iloc[test]
            
            rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
            feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

            # changed some values to numpy arrays by adding .values
            feat_selector.fit(feats_train.values, label_train.values)

            feats_train_ = feat_selector.transform(feats_train.values)
            select_feats = feat_selector.support_   # creates a boolean mask of used values in the training feats
            select_feats = np.argwhere(select_feats == True)
            feats_cv.extend(select_feats)
            
#             feats_train = feats_train.iloc[:,select_feats]
#             feats_test = feats_test.iloc[:,select_feats]
            feats_train = feats_train.values[:,select_feats]
            feats_test = feats_test.values[:,select_feats]
            
            feats_train = feats_train.reshape(feats_train.shape[0], feats_train.shape[1]) #reshape to 2D array
            feats_test = feats_test.reshape(feats_test.shape[0], feats_test.shape[1]) #reshape to 2D array

            feats_train_ = feats_train
            label_train_ = label_train

            feats_test_ = feats_test
            label_test_ = label_test
            
            print(feats_train.shape)
            model = rf.fit(feats_train,label_train)

            probs_test = model.predict_proba(feats_test_)
            fpr, tpr, thresholds = metrics.roc_curve(label_test_, probs_test[:,1], pos_label=1)
            auc_cv.append(metrics.auc(fpr,tpr))
    
    feats_cv = [x[0] for x in feats_cv] # Convert each NumPy array to a tuple
    counter = collections.Counter(feats_cv)
    top_feats_sorted = np.array(counter.most_common(5))
    top_feats = top_feats_sorted[:,0]

    final_feats = feats.iloc[:, top_feats]
    final_feats = final_feats#.to_numpy()
    label = label.to_numpy()

#     final_model = RandomForestClassifier(gamma='auto', probability=True)
    final_model = RandomForestClassifier()
    final_model.fit(final_feats,label)

    return statistics.mean(auc_cv), final_model, top_feats

In [ ]:
if __name__ == "__main__":
    
    # feats1 and label1 are for train, feats2 and label2 are for test
        
    feats1 = training_matrix
    feats2 = testing_matrix
    
    load_labels = loadmat(os.path.join("G:/.shortcut-targets-by-id/1YKJmC9bzTUqncCWUJdAy3ZSVLcEt10UD/KiTS Project/Files",'trainLabel.mat'))
    label1 = pd.Series((load_labels['trainLabel']).flatten())
#     label1 = trainISUP.reset_index(drop=True)
    
    load_labels = loadmat(os.path.join("G:/.shortcut-targets-by-id/1YKJmC9bzTUqncCWUJdAy3ZSVLcEt10UD/KiTS Project/Files",'testLabel.mat'))
    label2 = pd.Series((load_labels['testLabel']).flatten())
#     label2 = testISUP.reset_index(drop=True)
    
    cvAUC, cvModel, topFeats = cvClassify(feats1,label1)
    print("cross validation AUC is "+ str(cvAUC))
    print("Columns with the most commonly used features:" + str(topFeats))

In [ ]:
    # ValueError: X.shape[1] = 192 should be equal to 5, the number of features at training time
    # Am I supposed to set feats2 as only the most used columns?
#     feats2 = feats2.to_numpy()
#     label2 = label2.to_numpy()
    topFeatsMatrix = feats2.iloc[:, topFeats]
    probs_ = cvModel.predict_proba(topFeatsMatrix)

    fpr, tpr, thresholds = metrics.roc_curve(label2, probs_[:,1], pos_label=1)
    testAUC = metrics.auc(fpr,tpr)
    print("test AUC is "+ str(testAUC))
    pred0 = 0
    pred1 = 1
    for i in range(len(probs_)):
        print(probs_[i], ", label: ", label2[i])
        if label2[i] == 0:
            pred0+=1
        else:
            pred1+=1
    print(pred0)
    print(pred1)

In [10]:
import json

path = "/Users/dhruv/OneDrive - Georgia Institute of Technology/KiTS Project/RawData/case_00000/raw/meta.json"
meta = json.load(open(path))

In [11]:
meta

{'x_spacing': 0.919921875,
 'y_spacing': 0.919921875,
 'z_spacing': 0.5,
 'num_images': 611}

In [12]:
training_ids

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu May 12 16:15:18 2022',
 '__version__': '1.0',
 '__globals__': [],
 'trainingGroup': array([[183],
        [147],
        [274],
        [ 42],
        [ 52],
        [ 28],
        [276],
        [ 44],
        [229],
        [138],
        [267],
        [137],
        [219],
        [174],
        [164],
        [188],
        [257],
        [ 46],
        [  8],
        [176],
        [244],
        [ 77],
        [221],
        [  4],
        [144],
        [258],
        [204],
        [101],
        [ 21],
        [209],
        [203],
        [102],
        [167],
        [ 15],
        [149],
        [ 76],
        [154],
        [280],
        [ 40],
        [201],
        [226],
        [ 61],
        [300],
        [119],
        [ 88],
        [165],
        [268],
        [220],
        [227],
        [255],
        [116],
        [120],
        [107],
        [ 90],
        [104],
        [192],
    

In [ ]:
index=range(300)